In [1]:
import json
import re
from pathlib import Path
from typing import Dict, List, Tuple
from tqdm import tqdm
import random

## 1. Phân tích Strategy theo từng môn học

Mỗi môn học có cách suy luận khác nhau:

In [2]:
# Định nghĩa reasoning strategy cho từng môn

REASONING_STRATEGIES = {
    "Mathematics": {
        "description": "Toán học - Giải từng bước, công thức, tính toán",
        "steps": [
            "Xác định dạng bài toán và công thức cần dùng",
            "Liệt kê các dữ kiện đã cho",
            "Thực hiện các bước tính toán",
            "Kiểm tra và kết luận"
        ],
        "keywords": ["tính", "phương trình", "công thức", "biểu thức", "giải", "tìm", "đạo hàm", "tích phân"]
    },
    
    "Physics": {
        "description": "Vật lý - Phân tích hiện tượng, áp dụng định luật",
        "steps": [
            "Xác định hiện tượng vật lý và định luật liên quan",
            "Phân tích các đại lượng và mối quan hệ",
            "Áp dụng công thức hoặc định luật",
            "So sánh với các đáp án và kết luận"
        ],
        "keywords": ["định luật", "lực", "năng lượng", "chuyển động", "điện", "từ", "quang", "nhiệt"]
    },
    
    "Chemistry": {
        "description": "Hóa học - Phương trình, cân bằng, tính toán mol",
        "steps": [
            "Xác định chất tham gia và phản ứng hóa học",
            "Viết và cân bằng phương trình (nếu có)",
            "Tính toán theo mol, khối lượng hoặc thể tích",
            "Phân tích tính chất và kết luận"
        ],
        "keywords": ["phản ứng", "chất", "mol", "khối lượng", "dung dịch", "axit", "bazơ", "oxi hóa", "khử"]
    },
    
    "Biology": {
        "description": "Sinh học - Phân tích đặc điểm, chức năng sinh học",
        "steps": [
            "Xác định khái niệm hoặc quá trình sinh học",
            "Phân tích đặc điểm và chức năng",
            "So sánh các đáp án với kiến thức sinh học",
            "Loại trừ các đáp án sai và kết luận"
        ],
        "keywords": ["tế bào", "gen", "di truyền", "sinh sản", "hô hấp", "quang hợp", "tiến hóa", "sinh thái"]
    },
    
    "Geography": {
        "description": "Địa lý - Phân tích vị trí, đặc điểm địa lý",
        "steps": [
            "Xác định vùng địa lý hoặc hiện tượng",
            "Phân tích các yếu tố địa lý (khí hậu, địa hình, tài nguyên)",
            "So sánh với các đặc điểm trong đáp án",
            "Kết luận dựa trên kiến thức địa lý"
        ],
        "keywords": ["vùng", "khí hậu", "địa hình", "tài nguyên", "dân cư", "kinh tế", "môi trường"]
    },
    
    "History": {
        "description": "Lịch sử - Phân tích sự kiện, nguyên nhân, hậu quả",
        "steps": [
            "Xác định sự kiện lịch sử và thời gian",
            "Phân tích nguyên nhân và bối cảnh",
            "Xem xét hậu quả và ý nghĩa",
            "So sánh với các đáp án và kết luận"
        ],
        "keywords": ["sự kiện", "thời kỳ", "chiến tranh", "cách mạng", "phong trào", "triều đại", "năm"]
    },
    
    "Literature": {
        "description": "Ngữ văn - Phân tích văn học, tu từ, ý nghĩa",
        "steps": [
            "Xác định tác phẩm, tác giả hoặc đoạn văn",
            "Phân tích nội dung, hình thức và nghệ thuật",
            "Xem xét biện pháp tu từ và giá trị văn học",
            "Kết luận về ý nghĩa và thông điệp"
        ],
        "keywords": ["tác phẩm", "tác giả", "thơ", "văn xuôi", "biện pháp", "hình tượng", "chủ đề"]
    },
    
    "CivicEducation": {
        "description": "Giáo dục công dân - Phân tích luật pháp, đạo đức",
        "steps": [
            "Xác định khái niệm hoặc quy định pháp luật",
            "Phân tích quyền, nghĩa vụ và trách nhiệm",
            "So sánh với các giá trị đạo đức và xã hội",
            "Kết luận dựa trên chuẩn mực pháp lý và đạo đức"
        ],
        "keywords": ["quyền", "nghĩa vụ", "pháp luật", "hiến pháp", "đạo đức", "công dân", "xã hội"]
    },
    
    "English": {
        "description": "Tiếng Anh - Grammar, vocabulary, context",
        "steps": [
            "Identify the grammar rule or vocabulary context",
            "Analyze the sentence structure and meaning",
            "Evaluate each option against the context",
            "Select the most appropriate answer"
        ],
        "keywords": ["grammar", "tense", "vocabulary", "preposition", "phrase", "clause"]
    }
}

# Hiển thị strategies
for subject, strategy in REASONING_STRATEGIES.items():
    print(f"\n{'='*60}")
    print(f"📚 {subject}: {strategy['description']}")
    print(f"{'='*60}")
    for i, step in enumerate(strategy['steps'], 1):
        print(f"  {i}. {step}")


📚 Mathematics: Toán học - Giải từng bước, công thức, tính toán
  1. Xác định dạng bài toán và công thức cần dùng
  2. Liệt kê các dữ kiện đã cho
  3. Thực hiện các bước tính toán
  4. Kiểm tra và kết luận

📚 Physics: Vật lý - Phân tích hiện tượng, áp dụng định luật
  1. Xác định hiện tượng vật lý và định luật liên quan
  2. Phân tích các đại lượng và mối quan hệ
  3. Áp dụng công thức hoặc định luật
  4. So sánh với các đáp án và kết luận

📚 Chemistry: Hóa học - Phương trình, cân bằng, tính toán mol
  1. Xác định chất tham gia và phản ứng hóa học
  2. Viết và cân bằng phương trình (nếu có)
  3. Tính toán theo mol, khối lượng hoặc thể tích
  4. Phân tích tính chất và kết luận

📚 Biology: Sinh học - Phân tích đặc điểm, chức năng sinh học
  1. Xác định khái niệm hoặc quá trình sinh học
  2. Phân tích đặc điểm và chức năng
  3. So sánh các đáp án với kiến thức sinh học
  4. Loại trừ các đáp án sai và kết luận

📚 Geography: Địa lý - Phân tích vị trí, đặc điểm địa lý
  1. Xác định vùng địa 

## 2. Load và phân tích dữ liệu

In [3]:
# Load data
input_file = Path("../data/sft_dataset_vnhsge/train_sft.jsonl")

data = []
with open(input_file, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

print(f"Loaded {len(data)} samples")

# Phân tích phân phối môn học
from collections import Counter

subjects = [item['subject'] for item in data]
subject_counts = Counter(subjects)

print("\nSubject distribution:")
for subject, count in sorted(subject_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{subject:20s}: {count:4d} samples")

Loaded 1573 samples

Subject distribution:
English             :  250 samples
CivicEducation      :  200 samples
History             :  200 samples
Chemistry           :  196 samples
Biology             :  190 samples
Geography           :  190 samples
Mathematics         :  180 samples
Physics             :  167 samples


## 3. Các template reasoning theo môn học

Tạo templates cụ thể cho từng môn:

In [4]:
def generate_reasoning_template(subject: str, question: str, options: Dict[str, str], correct_answer: str) -> str:
    """
    Generate reasoning template based on subject
    
    Args:
        subject: Môn học
        question: Câu hỏi
        options: Dict của các options {"A": "...", "B": "...", ...}
        correct_answer: Đáp án đúng ("A", "B", "C", "D")
    
    Returns:
        Reasoning text với <think> tags
    """
    
    if subject == "Mathematics":
        reasoning = f"""<think>
Xác định dạng bài và công thức cần dùng
Phân tích dữ kiện và mối quan hệ
Áp dụng công thức và tính toán
Đáp án {correct_answer} phù hợp với kết quả
</think>"""
    
    elif subject == "Physics":
        reasoning = f"""<think>
Xác định hiện tượng vật lý và định luật liên quan
Phân tích các đại lượng và mối quan hệ
Áp dụng định luật và tính toán
Đáp án {correct_answer} đúng theo định luật vật lý
</think>"""
    
    elif subject == "Chemistry":
        reasoning = f"""<think>
Xác định chất tham gia và loại phản ứng
Phân tích tính chất hóa học
Viết và cân bằng phương trình (nếu cần)
Đáp án {correct_answer} phù hợp với tính chất/phản ứng
</think>"""
    
    elif subject == "Biology":
        reasoning = f"""<think>
Xác định khái niệm/quá trình sinh học
Phân tích cấu trúc và chức năng
So sánh các đáp án với kiến thức sinh học
Đáp án {correct_answer} chính xác
</think>"""
    
    elif subject == "Geography":
        reasoning = f"""<think>
Xác định vùng địa lý hoặc hiện tượng
Phân tích yếu tố địa lý (khí hậu, địa hình, tài nguyên)
So sánh với đặc điểm các đáp án
Đáp án {correct_answer} đúng
</think>"""
    
    elif subject == "History":
        reasoning = f"""<think>
Xác định sự kiện, thời kỳ và bối cảnh lịch sử
Phân tích nguyên nhân và diễn biến
Xem xét hậu quả và ý nghĩa
Đáp án {correct_answer} phù hợp với sự kiện lịch sử
</think>"""
    
    elif subject == "Literature":
        reasoning = f"""<think>
Xác định tác phẩm, tác giả và thể loại
Phân tích nội dung, nghệ thuật và biện pháp tu từ
Đánh giá giá trị văn học và thông điệp
Đáp án {correct_answer} phù hợp
</think>"""
    
    elif subject == "CivicEducation":
        reasoning = f"""<think>
Xác định khái niệm hoặc quy định pháp luật
Phân tích quyền, nghĩa vụ và trách nhiệm
So sánh với chuẩn mực pháp lý và đạo đức
Đáp án {correct_answer} đúng theo quy định
</think>"""
    
    elif subject == "English":
        reasoning = f"""<think>
Identify question type (grammar/vocabulary)
Analyze sentence structure and context
Evaluate each option against the rule
Answer {correct_answer} is correct
</think>"""
    
    else:
        # Generic template for unknown subjects
        reasoning = f"""<think>
Phân tích câu hỏi và yêu cầu
Xem xét và so sánh các đáp án
Loại trừ đáp án không phù hợp
Đáp án {correct_answer} là đáp án đúng
</think>"""
    
    return reasoning.strip()


# Test với một vài samples
print("\nTesting reasoning generation:\n")
test_sample = data[0]
print(f"Subject: {test_sample['subject']}")
print(f"Question: {test_sample['messages'][1]['content'][:100]}...")
print(f"\nGenerated reasoning preview (first 300 chars):")

# Parse question to get options
def parse_question(content):
    lines = content.strip().split('\n')
    question_parts = []
    options = {}
    expected_labels = ['A', 'B', 'C', 'D']
    current_label_idx = 0
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        
        # Match cả có và không có dấu chấm: A. text hoặc A text
        match = re.match(r'^([A-D])[\.\:\s]*\s*(.*)', line)
        if match:
            label = match.group(1)
            text = match.group(2).strip()
            
            # Skip nếu text rỗng
            if not text:
                continue
            
            # Auto-fix: Luôn dùng label theo thứ tự expected
            if current_label_idx < len(expected_labels):
                actual_label = expected_labels[current_label_idx]
                
                if label in options or label != actual_label:
                    # print(f"⚠️ Warning: Found '{label}', using '{actual_label}' instead")
                    label = actual_label
                
                options[label] = text
                current_label_idx += 1
        else:
            # Nếu không phải option, coi là phần question
            question_parts.append(line)
    
    return ' '.join(question_parts), options

q, opts = parse_question(test_sample['messages'][1]['content'])
correct = json.loads(test_sample['messages'][2]['content'])['answer']
reasoning = generate_reasoning_template(test_sample['subject'], q, opts, correct)
print(reasoning)


Testing reasoning generation:

Subject: Biology
Question: Câu 81: Có thể sử dụng hóa chất nào sau đây để phát hiện quá trình hô hấp ở thực vật thải ra khí CO2...

Generated reasoning preview (first 300 chars):
<think>
Xác định khái niệm/quá trình sinh học
Phân tích cấu trúc và chức năng
So sánh các đáp án với kiến thức sinh học
Đáp án B chính xác
</think>


## 4. Augment dataset với reasoning

In [5]:
def add_reasoning_to_dataset(input_file: Path, output_file: Path, sample_ratio: float = 1.0):
    """
    Thêm reasoning vào dataset
    
    Args:
        input_file: File JSONL đầu vào
        output_file: File JSONL đầu ra
        sample_ratio: Tỷ lệ samples cần thêm reasoning (0.0-1.0)
    """
    # Load data
    data = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    
    print(f"Loaded {len(data)} samples")
    
    # Process data
    augmented_data = []
    skipped = 0
    
    for item in tqdm(data, desc="Adding reasoning"):
        try:
            # Parse question
            user_content = item['messages'][1]['content']
            question, options = parse_question(user_content)
            
            if len(options) != 4:
                # Giữ nguyên nếu không parse được
                augmented_data.append(item)
                print(item)
                skipped += 1
                continue
            
            # Get correct answer
            correct_answer = json.loads(item['messages'][2]['content'])['answer']
            
            # Decide whether to add reasoning
            if random.random() > sample_ratio:
                # Giữ nguyên
                augmented_data.append(item)
                continue
            
            # Generate reasoning
            reasoning = generate_reasoning_template(
                subject=item['subject'],
                question=question,
                options=options,
                correct_answer=correct_answer
            )
            
            # ✅ NEW FORMAT: reasoning nằm ngoài JSON
            # Format: <think>...</think>\n{"answer":"A"}
            answer_json = json.dumps({"answer": correct_answer}, ensure_ascii=False)
            new_response_content = f"{reasoning}\n{answer_json}"
            
            # Create new item
            new_item = {
                "messages": [
                    item['messages'][0],  # System message
                    item['messages'][1],  # User message
                    {
                        "role": "assistant",
                        "content": new_response_content
                    }
                ],
                "id": item['id'],
                "subject": item['subject']
            }
            
            augmented_data.append(new_item)
        
        except Exception as e:
            print(f"\nError processing {item.get('id', 'unknown')}: {e}")
            augmented_data.append(item)  # Giữ nguyên nếu lỗi
            print(item)
            skipped += 1
    
    print(f"\nProcessed {len(augmented_data)} samples")
    print(f"Skipped: {skipped}")
    print(f"Added reasoning to: {len(augmented_data) - skipped} samples")
    
    # Save
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in augmented_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    
    print(f"\nSaved to {output_file}")
    return augmented_data

In [6]:
# Thực hiện augmentation
output_file = Path("../data/sft_dataset_vnhsge/train_sft_with_reasoning.jsonl")

augmented_data = add_reasoning_to_dataset(
    input_file=input_file,
    output_file=output_file,
    sample_ratio=1.0  # 1.0 = thêm reasoning cho 100% samples, 0.5 = 50%, etc.
)

Loaded 1573 samples


Adding reasoning: 100%|██████████| 1573/1573 [00:00<00:00, 74379.00it/s]


Processed 1573 samples
Skipped: 0
Added reasoning to: 1573 samples

Saved to ..\data\sft_dataset_vnhsge\train_sft_with_reasoning.jsonl


## 5. Kiểm tra kết quả

In [7]:
# Kiểm tra một vài samples
print("\n" + "="*80)
print("Sample data with reasoning:")
print("="*80)

# Group by subject
from collections import defaultdict
samples_by_subject = defaultdict(list)
for item in augmented_data:
    samples_by_subject[item['subject']].append(item)

# Show one sample per subject
for subject in ["Mathematics", "Physics", "Chemistry", "Biology"]:
    if subject in samples_by_subject:
        sample = samples_by_subject[subject][0]
        print(f"\n{'='*80}")
        print(f"📚 Subject: {subject}")
        print(f"ID: {sample['id']}")
        print(f"{'='*80}")
        print(f"\n📝 Question:\n{sample['messages'][1]['content'][:200]}...")
        print(f"\n🤔 Assistant Response:")
        response_content = sample['messages'][2]['content']
        
        # Check if response has reasoning (contains <think>)
        if '<think>' in response_content:
            print(response_content[:500] + "...")
        else:
            # Old format or no reasoning
            try:
                response = json.loads(response_content)
                print(f"Answer: {response['answer']}")
            except:
                print(response_content[:200])


Sample data with reasoning:

📚 Subject: Mathematics
ID: MET_Math_IE_2019_1

📝 Question:
Câu 1) Thể tích của khối lập phương cạnh 2a bằng:
A. 8a^3. 
B. 2a^3. 
C. a^3.   
D. 6a^3....

🤔 Assistant Response:
<think>
Xác định dạng bài và công thức cần dùng
Phân tích dữ kiện và mối quan hệ
Áp dụng công thức và tính toán
Đáp án A phù hợp với kết quả
</think>
{"answer": "A"}...

📚 Subject: Physics
ID: MET_Phy_IE_2019_1

📝 Question:
Câu 1. 	Một vật dao động điều hoà theo phương trình: 
x = Acos(\omega*t+\varphi) với A > 0 và omega > 0. Pha của dao động ở thời điểm t là 
A. \omega
B. cos(\omega*t+ \varphi)
C. \omega*t+ \varphi
D. ...

🤔 Assistant Response:
<think>
Xác định hiện tượng vật lý và định luật liên quan
Phân tích các đại lượng và mối quan hệ
Áp dụng định luật và tính toán
Đáp án C đúng theo định luật vật lý
</think>
{"answer": "C"}...

📚 Subject: Chemistry
ID: MET_Chem_IE_2019_1

📝 Question:
Câu 41: Ở điều kiện thường, kim loại nào sau đây ở trạng thái lỏng?
A. ${Zn}$.
B. ${Hg}$.
C. $

## 6. Cập nhật system prompt

Khi training với reasoning, cần cập nhật system prompt:

In [8]:
# System prompt mới
NEW_SYSTEM_PROMPT = """Bạn là trợ lý trả lời trắc nghiệm thông minh. 
Hãy suy luận từng bước trong thẻ <think>, sau đó trả JSON với đáp án.
Format: 
<think>Bước 1: ...
Bước 2: ...
Kết luận: ...</think>
{"answer":"A"}"""

print("New system prompt:")
print(NEW_SYSTEM_PROMPT)
print("\n" + "="*80)

# Update system message in all samples
def update_system_prompt(data, new_prompt):
    for item in data:
        item['messages'][0]['content'] = new_prompt
    return data

# Apply
augmented_data = update_system_prompt(augmented_data, NEW_SYSTEM_PROMPT)

# Save again
with open(output_file, 'w', encoding='utf-8') as f:
    for item in augmented_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"Updated system prompt and saved to {output_file}")

New system prompt:
Bạn là trợ lý trả lời trắc nghiệm thông minh. 
Hãy suy luận từng bước trong thẻ <think>, sau đó trả JSON với đáp án.
Format: 
<think>Bước 1: ...
Bước 2: ...
Kết luận: ...</think>
{"answer":"A"}

Updated system prompt and saved to ..\data\sft_dataset_vnhsge\train_sft_with_reasoning.jsonl


## 7. Statistics và phân tích

In [9]:
# Tính toán statistics
import numpy as np

reasoning_lengths = []
total_lengths = []

for item in augmented_data:
    response_content = item['messages'][2]['content']
    
    # Check if response has reasoning (new format)
    if '<think>' in response_content:
        # Extract reasoning part (from <think> to </think>)
        import re
        match = re.search(r'<think>.*?</think>', response_content, re.DOTALL)
        if match:
            reasoning_text = match.group(0)
            reasoning_lengths.append(len(reasoning_text))
            total_lengths.append(len(item['messages'][1]['content']) + len(reasoning_text))

print(f"Samples with reasoning: {len(reasoning_lengths)}")
print(f"\nReasoning length statistics:")
if reasoning_lengths:
    print(f"  Mean: {np.mean(reasoning_lengths):.0f} characters")
    print(f"  Median: {np.median(reasoning_lengths):.0f} characters")
    print(f"  Min: {np.min(reasoning_lengths):.0f} characters")
    print(f"  Max: {np.max(reasoning_lengths):.0f} characters")

    print(f"\nTotal length (question + reasoning) statistics:")
    print(f"  Mean: {np.mean(total_lengths):.0f} characters")
    print(f"  Max: {np.max(total_lengths):.0f} characters")

    # Ước tính tokens (roughly 1 token ~ 4 characters for Vietnamese)
    avg_tokens = np.mean(total_lengths) / 4
    print(f"\nEstimated average tokens: ~{avg_tokens:.0f} tokens")
else:
    print("No samples with reasoning found")

Samples with reasoning: 1573

Reasoning length statistics:
  Mean: 159 characters
  Median: 157 characters
  Min: 147 characters
  Max: 172 characters

Total length (question + reasoning) statistics:
  Mean: 516 characters
  Max: 3272 characters

Estimated average tokens: ~129 tokens


## 8. Tạo mixed dataset (có và không có reasoning)

Có thể tạo dataset kết hợp để model học cả hai cách:

In [10]:
# Tạo mixed dataset: 50% có reasoning, 50% không có
mixed_output = Path("../data/sft_dataset_vnhsge/train_sft_mixed_reasoning.jsonl")

print("Creating mixed dataset (50% with reasoning, 50% without)...")
augmented_mixed = add_reasoning_to_dataset(
    input_file=input_file,
    output_file=mixed_output,
    sample_ratio=0.5  # 50% có reasoning
)

# Count samples with reasoning (new format)
with_reasoning = sum(1 for item in augmented_mixed 
                     if '<think>' in item['messages'][2]['content'])
print(f"\nMixed dataset statistics:")
print(f"  Total samples: {len(augmented_mixed)}")
print(f"  With reasoning: {with_reasoning} ({with_reasoning/len(augmented_mixed)*100:.1f}%)")
print(f"  Without reasoning: {len(augmented_mixed)-with_reasoning} ({(len(augmented_mixed)-with_reasoning)/len(augmented_mixed)*100:.1f}%)")

Creating mixed dataset (50% with reasoning, 50% without)...
Loaded 1573 samples


Adding reasoning: 100%|██████████| 1573/1573 [00:00<00:00, 56284.25it/s]


Processed 1573 samples
Skipped: 0
Added reasoning to: 1573 samples

Saved to ..\data\sft_dataset_vnhsge\train_sft_mixed_reasoning.jsonl

Mixed dataset statistics:
  Total samples: 1573
  With reasoning: 786 (50.0%)
  Without reasoning: 787 (50.0%)


## Tổng kết

**Files đã tạo:**
1. `train_sft_with_reasoning.jsonl` - 100% samples có reasoning
2. `train_sft_mixed_reasoning.jsonl` - 50% có reasoning, 50% không

**Lợi ích:**
- Model học cách suy luận có cấu trúc
- Tăng tính giải thích được
- Có thể cải thiện accuracy nhờ reasoning steps

**Lưu ý khi training:**
- Cần tăng `max_seq_length` vì có thêm reasoning
- Có thể cần điều chỉnh `batch_size` do sequence dài hơn
- Nên dùng `completion_only_loss=True` để chỉ tính loss trên reasoning + answer